# Forest Fires

The topic is predicting forest fires in two different regions of Algeria, namely the Bejaia region in northeast Algeria and the Siddi Bel-Abbes located in the northwest region. Our task would be to predict whether or not there will be a fire and we will base our analysis on different measurements: Temperature, Relative humidity, Wind speed, and Amount of rain. Other Fire Weather Index (FWI) measurements include: FWI

In [1]:
library(tidyverse)
library(repr)
library(readxl)
library(RColorBrewer)
library(forcats)
library(tidymodels)
options(repr.matrix.max.rows = 6) #limits output of dataframes to 6 rows

── Attaching packages ─────────────────────────────────────── tidyverse 1.3.0 ──

✔ ggplot2 3.3.2     ✔ purrr   0.3.4
✔ tibble  3.0.3     ✔ dplyr   1.0.2
✔ tidyr   1.1.2     ✔ stringr 1.4.0
✔ readr   1.3.1     ✔ forcats 0.5.0

Warning message:
“package ‘ggplot2’ was built under R version 4.0.1”
Warning message:
“package ‘tibble’ was built under R version 4.0.2”
Warning message:
“package ‘tidyr’ was built under R version 4.0.2”
Warning message:
“package ‘dplyr’ was built under R version 4.0.2”
── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()

Warning message:
“package ‘tidymodels’ was built under R version 4.0.2”
── Attaching packages ────────────────────────────────────── tidymodels 0.1.1 ──

✔ broom     0.7.0      ✔ recipes   0.1.13
✔ dials     0.0.9      ✔ rsample   0.0.7 
✔ infer     0.5.4      ✔ tune      0.1.1 
✔ modeldata 0.0.2      ✔ workflows 0.2.0 
✔ parsnip   0.1.3    

In [2]:
Bej_data1 <- read_csv("data/forest_fires.csv", skip = 1, n_max = 122)
SB_data1 <- read_csv("data/forest_fires.csv", skip = 126)
forest_fires1 <- rbind(Bej_data1, SB_data1)
forest_fires1 <- cbind(region = rep(c("Bejaia","Sidi-Bel Abbes"), each = 122), forest_fires1) %>% 
    mutate(Classes = as_factor(Classes))

forest_fires1

Parsed with column specification:
cols(
  day = col_double(),
  month = col_double(),
  year = col_double(),
  Temperature = col_double(),
  RH = col_double(),
  Ws = col_double(),
  Rain = col_double(),
  FFMC = col_double(),
  DMC = col_double(),
  DC = col_double(),
  ISI = col_double(),
  BUI = col_double(),
  FWI = col_double(),
  Classes = col_character()
)

Parsed with column specification:
cols(
  day = col_double(),
  month = col_double(),
  year = col_double(),
  Temperature = col_double(),
  RH = col_double(),
  Ws = col_double(),
  Rain = col_double(),
  FFMC = col_double(),
  DMC = col_double(),
  DC = col_double(),
  ISI = col_double(),
  BUI = col_double(),
  FWI = col_double(),
  Classes = col_character()
)



region         day month year Temperature RH Ws Rain FFMC DMC DC   ISI BUI
1   Bejaia         1   6     2012 29          57 18  0.0 65.7 3.4 7.6  1.3 3.4
2   Bejaia         2   6     2012 29          61 13  1.3 64.4 4.1 7.6  1.0 3.9
3   Bejaia         3   6     2012 26          82 22 13.1 47.1 2.5 7.1  0.3 2.7
⋮   ⋮              ⋮   ⋮     ⋮    ⋮           ⋮  ⋮  ⋮    ⋮    ⋮   ⋮    ⋮   ⋮  
242 Sidi-Bel Abbes 28  9     2012 27          87 29 0.5  45.9 3.5  7.9 0.4 3.4
243 Sidi-Bel Abbes 29  9     2012 24          54 18 0.1  79.7 4.3 15.2 1.7 5.1
244 Sidi-Bel Abbes 30  9     2012 24          64 15 0.2  67.3 3.8 16.5 1.2 4.8
    FWI Classes 
1   0.5 not fire
2   0.4 not fire
3   0.1 not fire
⋮   ⋮   ⋮       
242 0.2 not fire
243 0.7 not fire
244 0.5 not fire

In [22]:
url <- "https://archive.ics.uci.edu/ml/machine-learning-databases/00547/Algerian_forest_fires_dataset_UPDATE.csv"
Bej_data2 <- read_csv(url, skip = 1, n_max = 122)
SB_data2 <- read_csv(url, skip = 126)
forest_fires2 <- rbind(Bej_data2, SB_data2)
forest_fires2 <- cbind(region = rep(c("Bejaia","Sidi-Bel Abbes"), each = 122), forest_fires2) %>% 
    mutate(Classes = as_factor(Classes), 
            month = as.numeric(month),
            day = as.numeric(day)) #%>% 
#     separate(col = DC,
#             into = c("DC", "ISI"),
#             sep = " ")




Parsed with column specification:
cols(
  day = col_character(),
  month = col_character(),
  year = col_double(),
  Temperature = col_double(),
  RH = col_double(),
  Ws = col_double(),
  Rain = col_double(),
  FFMC = col_double(),
  DMC = col_double(),
  DC = col_double(),
  ISI = col_double(),
  BUI = col_double(),
  FWI = col_double(),
  Classes = col_character()
)

Parsed with column specification:
cols(
  day = col_character(),
  month = col_character(),
  year = col_double(),
  Temperature = col_double(),
  RH = col_double(),
  Ws = col_double(),
  Rain = col_double(),
  FFMC = col_double(),
  DMC = col_double(),
  DC = col_character(),
  ISI = col_double(),
  BUI = col_double(),
  FWI = col_character(),
  Classes = col_character()
)

Warning message:
“1 parsing failure.
row col   expected     actual                                                                                                       file
 44  -- 14 columns 13 columns 'https://archive.ics.uci.edu/ml/machine-lear

region         day month year Temperature RH Ws Rain FFMC DMC  DC     ISI 
1 Sidi-Bel Abbes 12  7     2012 36          44 13 0.0  90.1 12.6 19.4    8.3
2 Sidi-Bel Abbes 13  7     2012 39          45 13 0.6  85.2 11.3 10.4    4.2
3 Sidi-Bel Abbes 14  7     2012 37          37 18 0.2  88.9 12.9 14.6 9 12.5
  BUI  FWI  Classes
1 12.5 9.6  fire   
2 10.9 4.7  fire   
3 10.4 fire NA

[1] FALSE